In [55]:
#### IMPORTS

import re
import abc
from collections import Counter, defaultdict, namedtuple, deque, abc
from itertools   import (permutations, combinations, chain, cycle, product, islice, 
                         takewhile, zip_longest, count as count_from)
from functools   import lru_cache, reduce
from heapq import (heappush, heappop, nlargest, nsmallest)

from pprint import pprint as p, pformat as pf
import toolz.curried as t
from tqdm import tqdm_notebook as tq
from dataclasses import dataclass

#### CONSTANTS

alphabet = 'abcdefghijklmnopqrstuvwxyz'
ALPHABET = alphabet.upper()
infinity = float('inf')

#### SIMPLE UTILITY FUNCTIONS

cat = ''.join

def ints(start, end, step=1):
    "The integers from start to end, inclusive: range(start, end+1)"
    return range(start, end + 1, step)

def first(iterable, default=None): 
    "The first item in an iterable, or default if it is empty."
    return next(iter(iterable), default)

def head(iterable, n=5):
    "The first n items in an iterable"
    return tuple(islice(iterable, n))

def tail(iterable, n=1):
    "Skip n items in an iterable"
    return islice(iterable, n, None)

def first_true(iterable, pred=None, default=None):
    """Returns the first true value in the iterable.
    If no true value is found, returns *default*
    If *pred* is not None, returns the first item
    for which pred(item) is true."""
    # first_true([a,b,c], default=x) --> a or b or c or x
    # first_true([a,b], fn, x) --> a if fn(a) else b if fn(b) else x
    return next(filter(pred, iterable), default)

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

def upto(iterable, maxval):
    "From a monotonically increasing iterable, generate all the values <= maxval."
    # Why <= maxval rather than < maxval? In part because that's how Ruby's upto does it.
    return takewhile(lambda x: x <= maxval, iterable)

identity = lambda x: x

def quantify(iterable, pred=bool):
    "Count how many times the predicate is true of an item in iterable."
    return sum(map(pred, iterable))

def multimap(items):
    "Given (key, val) pairs, return {key: [val, ....], ...}."
    result = defaultdict(list)
    for (key, val) in items:
        result[key].append(val)
    return result

def overlapping(iterable, n):
    """Generate all (overlapping) n-element subsequences of iterable.
    overlapping('ABCDEFG', 3) --> ABC BCD CDE DEF EFG"""
    if isinstance(iterable, abc.Sequence):
        yield from (iterable[i:i+n] for i in range(len(iterable) + 1 - n))
    else:
        result = deque(maxlen=n)
        for x in iterable:
            result.append(x)
            if len(result) == n:
                yield tuple(result)
                
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    return overlapping(iterable, 2)

def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

def map2d(fn, grid):
    "Apply fn to every element in a 2-dimensional grid."
    return tuple(mapt(fn, row) for row in grid)

def flatmap(fn, *args):
    "Do a map and a one-level flatten"
    return tuple(chain.from_iterable(map(fn, *args)))

def repeat(n, fn, arg, *args, **kwds):
    "Repeat arg = fn(arg) n times, return arg."
    return nth(repeatedly(fn, arg, *args, **kwds), n)

def repeatedly(fn, arg, *args, **kwds):
    "Yield arg, fn(arg), fn(fn(arg)), ..."
    yield arg
    while True:
        arg = fn(arg, *args, **kwds)
        yield arg
        
def repeatedly1(fn, arg, *args, **kwds):
    "Yield fn(arg), fn(fn(arg)), ..."
    return tail(repeatedly(fn, arg, *args, **kwds))

def compose(f, g): 
    "The function that computes f(g(x))."
    return lambda x: f(g(x))

#### FILE INPUT AND PARSING

def Input(day, line_parser=str.strip, test=False, file_template='data/2019/{}.txt'):
    "For this day's input file, return a tuple of each line parsed by `line_parser`."
    return mapt(line_parser, open(file_template.format(
        f'{day}test' if test else day
    )))

@t.curry
def Tokens(line, sep=','):
    "Splits line into delimited tokens"
    return line.strip().split(sep)

def integers(text): 
    "A tuple of all integers in a string (ignore other characters)."
    return mapt(int, re.findall(r'-?\d+', text))

################ 2-D points implemented using (x, y) tuples

def X(point): return point[0]
def Y(point): return point[1]

origin = (0, 0)
HEADINGS = UP, LEFT, DOWN, RIGHT = (0, -1), (-1, 0), (0, 1), (1, 0)

def turn_right(heading): return HEADINGS[HEADINGS.index(heading) - 1]
def turn_around(heading):return HEADINGS[HEADINGS.index(heading) - 2]
def turn_left(heading):  return HEADINGS[HEADINGS.index(heading) - 3]

def add(A, B): 
    "Element-wise addition of two n-dimensional vectors."
    return mapt(sum, zip(A, B))

def neighbors4(point): 
    "The four neighboring squares."
    x, y = point
    return (          (x, y-1),
            (x-1, y),           (x+1, y), 
                      (x, y+1))

def neighbors8(point): 
    "The eight neighboring squares."
    x, y = point 
    return ((x-1, y-1), (x, y-1), (x+1, y-1),
            (x-1, y),             (x+1, y),
            (x-1, y+1), (x, y+1), (x+1, y+1))

def cityblock_distance(P, Q=origin): 
    "Manhatten distance between two points."
    return sum(abs(p - q) for p, q in zip(P, Q))

def distance(P, Q=origin): 
    "Straight-line (hypotenuse) distance between two points."
    return sum((p - q) ** 2 for p, q in zip(P, Q)) ** 0.5

def king_distance(P, Q=origin):
    "Number of chess King moves between two points."
    return max(abs(p - q) for p, q in zip(P, Q))

################ Debugging 

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def grep(pattern, iterable):
    "Print lines from iterable that match pattern."
    for line in iterable:
        if re.search(pattern, line):
            print(line)
            
class Struct:
    "A structure that can have any fields defined."
    def __init__(self, **entries): self.__dict__.update(entries)
    def __repr__(self): 
        fields = ['{}={}'.format(f, self.__dict__[f]) 
                  for f in sorted(self.__dict__)]
        return 'Struct({})'.format(', '.join(fields))

################ A* and Breadth-First Search (tracking states, not actions)

def always(value): return (lambda *args: value)

def Astar(start, moves_func, h_func, cost_func=always(1)):
    "Find a shortest sequence of states from start to a goal state (where h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    Path      = lambda s: ([] if (s is None) else Path(previous[s]) + [s])
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(s)
        for s2 in moves_func(s):
            g = path_cost[s] + cost_func(s, s2)
            if s2 not in path_cost or g < path_cost[s2]:
                heappush(frontier, (g + h_func(s2), s2))
                path_cost[s2] = g
                previous[s2] = s

def bfs(start, moves_func, goals):
    "Breadth-first search"
    goal_func = (goals if callable(goals) else lambda s: s in goals)
    return Astar(start, moves_func, lambda s: (0 if goal_func(s) else 1))

## Day 1

In [3]:
masses = Input(1, line_parser=int)
sum(m // 3 - 2 for m in masses)

3167282

In [4]:
def to_fuel(n):
    return n // 3 - 2

def total_fuel(mass):
    return (
        sum(takewhile(lambda n: n > 0,
                      repeatedly(to_fuel, mass)))
        - mass)

sum(map(total_fuel, masses))

4748063

## Day 2

In [43]:
orig = list(Input(2, line_parser=integers)[0])
# prog = [1,9,10,3,2,3,11,0,99,30,40,50]
prog = list(orig)
prog[1] = 12
prog[2] = 2
len(prog)

145

In [44]:
def process(pos, prog=prog):
    opcode, reg1, reg2, out = prog[pos:pos+4]
    if opcode == 99:
        return -1
    if opcode == 1:
        prog[out] = prog[reg1] + prog[reg2]
    if opcode == 2:
        prog[out] = prog[reg1] * prog[reg2]
    return pos + 4
first_true(repeatedly(process, 0), pred=lambda x: x is -1)

-1

In [45]:
prog[0]

8017076

In [49]:
def trial(noun, verb, prog=prog):
    prog = list(prog)
    prog[1] = noun
    prog[2] = verb
    pos = 0
    while pos != -1:
        pos = process(pos, prog)
    return prog[0]

goal = 19690720
first((noun, verb)
      for noun in range(100)
      for verb in range(100)
      if trial(noun, verb, orig) == goal)

(31, 46)

## Day 3

In [87]:
def Path(line):
    return [(token[0], int(token[1:])) for token in Tokens(line)]

path1, path2 = Input(3, line_parser=Path)
# path1 = Path('R8,U5,L5,D3')
# path2 = Path('U7,R6,D4,L4')
path1[:5]

[('R', 1000), ('D', 940), ('L', 143), ('D', 182), ('L', 877)]

In [88]:
def along(section, start=origin):
    direction, length = section
    heading = HEADINGS['ULDR'.index(direction)]
    return head(repeatedly1(add, start, heading), n=length)

def step(walk, section):
    start = walk[-1] if walk else origin
    return (*walk, *along(section, start))

step([], ('R', 8))

((1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0))

In [89]:
walk1 = reduce(step, path1, [])
walk2 = reduce(step, path2, [])
intersections = set(walk1) & set(walk2)
min(mapt(cityblock_distance, intersections))

865

In [92]:
def wire_distance(intersection):
    # Add 2 since we need to account for the origin for two paths
    return walk1.index(intersection) + walk2.index(intersection) + 2

min(mapt(wire_distance, intersections))

35038

## Day 4

In [97]:
my_input = '284639-748759'
my_range = range(*mapt(int, Tokens('284639-748759', sep='-')))
my_range

range(284639, 748759)

In [133]:
def digits(number):
    return mapt(int, str(number))

def has_repeat(digs):
    return any(i == j for i, j in pairwise(digs))

def is_monotonic(digs):
    return all(i <= j for i, j in pairwise(digs))

def is_password(number):
    digs = digits(number)
    return is_monotonic(digs) and has_repeat(digs)

quantify(my_range, is_password)

895

The monotonic condition ensures that all unique digit values appear together.

In [135]:
def has_single_repeat(digs):
    return any(count == 2 for count in Counter(digs).values())

def is_paironly_password(number):
    digs = digits(number)
    return is_monotonic(digs) and has_single_repeat(digs)

quantify(my_range, is_paironly_password)

591

## Day 5

In [12]:
prog, = Input(5, line_parser=integers)
prog[:5]

(3, 225, 1, 225, 6)

In [13]:
ADD, MUL, INPUT, OUTPUT, JT, JF, LT, EQ, STOP = (
    1, 2, 3, 4, 5, 6, 7, 8, 99)

arities = {
    ADD: 3, MUL: 3, INPUT: 1, OUTPUT: 1,
    JT: 2, JF: 2, LT: 3, EQ: 3,
    STOP: 0
}

def Opcode(val):
    code = val % 100
    arity = arities[code]
    return (val % 100,
            [val // 100 % 10, val // 1000 % 10, val // 10000 % 10][:arity])
Opcode(1003)

(3, [0])

In [18]:
def read_arg(prog, pc, mode):
    "Reads in argument from prog accounting for mode."
    arg = prog[pc]
    output = prog[arg] if mode == 0 else arg
    return output

read_arg([1002, 4,3,4,33], 1, 0)

33

In [19]:
def run(prog, inputs):
    prog = list(prog)
    inputs = iter(inputs)
    outputs = []
    pc = 0
    while True:
        code, modes = Opcode(prog[pc])
#         print(f'{prog[pc:pc+len(modes) + 1]}')
        if code == STOP:
            return outputs[-1]
        if code == ADD:
            arg1 = read_arg(prog, pc + 1, modes[0])
            arg2 = read_arg(prog, pc + 2, modes[1])
            out = prog[pc + 3]
            prog[out] = arg1 + arg2
        elif code == MUL:
            arg1 = read_arg(prog, pc + 1, modes[0])
            arg2 = read_arg(prog, pc + 2, modes[1])
            out = prog[pc + 3]
            prog[out] = arg1 * arg2
        elif code == INPUT:
            out = prog[pc + 1]
            prog[out] = next(inputs)
        elif code == OUTPUT:
            val = read_arg(prog, pc + 1, modes[0])
            outputs.append(val)
        elif code == JT:
            arg1 = read_arg(prog, pc + 1, modes[0])
            arg2 = read_arg(prog, pc + 2, modes[1])
            if arg1 != 0:
                pc = arg2
                continue
        elif code == JF:
            arg1 = read_arg(prog, pc + 1, modes[0])
            arg2 = read_arg(prog, pc + 2, modes[1])
            if arg1 == 0:
                pc = arg2
                continue
        elif code == LT:
            arg1 = read_arg(prog, pc + 1, modes[0])
            arg2 = read_arg(prog, pc + 2, modes[1])
            out = prog[pc + 3]
            prog[out] = int(arg1 < arg2)
        elif code == EQ:
            arg1 = read_arg(prog, pc + 1, modes[0])
            arg2 = read_arg(prog, pc + 2, modes[1])
            out = prog[pc + 3]
            prog[out] = int(arg1 == arg2)
        else:
            raise ValueError(f'Bad opcode at pc: {pc}')
        pc += len(modes) + 1
run([3,9,7,9,10,9,4,9,99,-1,8], [6])

1

In [23]:
assert run([3,0,4,0,99], [10]) == 10
assert run([3,0,4,0,99], [10]) == 10
assert run([3,9,8,9,10,9,4,9,99,-1,8], [10]) == 0
assert run([3,9,8,9,10,9,4,9,99,-1,8], [8]) == 1
assert run([3,9,7,9,10,9,4,9,99,-1,8], [6]) == 1
assert run([3,9,7,9,10,9,4,9,99,-1,8], [8]) == 0
assert run([3,3,1108,-1,8,3,4,3,99], [8]) == 1
assert run([3,3,1107,-1,8,3,4,3,99], [8]) == 0

In [21]:
run(prog, [1])

14522484

In [24]:
run(prog, [5])

4655956

## Day 6

In [42]:
edges = Input(6, line_parser=Tokens(sep=')'), test=True)
edges[:3]

(['COM', 'B'], ['B', 'C'], ['C', 'D'])

In [40]:
def add_edge(tree, edge):
    parent, child = edge
    tree[parent].append(child)
    return tree

def Tree(edges):
    return reduce(add_edge, edges, defaultdict(list))

In [41]:
def sum_depths(tree, node='COM', depth=0):
    children = tree[node]
    if not children:
        return depth
    return (depth
            + sum(sum_depths(tree, child, depth + 1) for child in children))
sum_depths(Tree(edges))

268504

The tree distance from A to B is the sum of their distances to their closest common ancestor.

In [57]:
tree = Tree(edges)
bfs('COM', lambda n: tree[n], ['YOU'])

['COM', 'B', 'C', 'D', 'E', 'J', 'K', 'YOU']